# Data Explorer

This notebook visualizes what is going on in the dataset. 

Each `.aedat4` file contains an event stream that contains event batches.
Event batches contains events that occured in each frame. 
The number of events in each batch can vary, especially for each letter and each subject.

This notebook aims to help us understand our dataset better and how the authors collected it.
It will assist us in passing the data as input spikes to the spiking neural network we created.

#### Helpful References:
* [IniVation DV-Processing API Documentation](https://dv-processing.inivation.com/rel_1_7/api.html#api)
   * Particularly, the [`Accumulator` class](https://dv-processing.inivation.com/rel_1_7/api.html#_CPPv4N2dv11AccumulatorE) is helpful in explaining how events are stored in event batches

## 0 - Choose Subject and Letter to Read

In [42]:
subject = 1
letter = 'a'


CSV = f"../data/csv/subject{subject}/{letter}_events.csv"
AEDAT = f"../data/aedat/subject{subject}/{letter}.aedat4"

## 1 - Explore Converted AEDAT4 Data (CSV)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np

In [35]:
df = pd.read_csv(CSV, index_col=False)
df

,Unnamed: 0,timestamp,x,y,polarity
0,0,324804107,97,49,0
1,1,324804107,110,49,0
2,2,324804107,108,52,0
3,3,324804238,112,102,0
4,4,324804303,102,120,0
...,...,...,...,...,...
1857593,50,351782070,160,144,0
1857594,51,351782648,170,124,0
1857595,52,351783075,210,11,0
1857596,53,351783155,214,12,0


In [36]:
min_length = np.inf
where = 0
counter = 0
samples = 0
stop = False # Use to stop at end of 1 sample? Or events? 

for i, event in enumerate(df["Unnamed: 0"]):
    if stop: break
    if event == 0 and counter > 0:
        # stop = True
        # Check if min_length is still minimum
        if min_length > counter:
            min_length = counter
            where = i

        counter = 0   # Restart counter
        samples += 1  # Increment sample


    counter += 1

print("Min length = ", min_length )
print("Number of samples = ", samples)
print("Where? ", where)

Min length =  9
Number of samples =  2662
Where?  901809


In [37]:
df[where - 2: where + 2]

,Unnamed: 0,timestamp,x,y,polarity
901807,7,338514746,138,118,0
901808,8,338520361,139,64,1
901809,0,338520645,101,14,1
901810,1,338521219,25,17,1


## 2 - Explore AEDAT4 Data

In [31]:
import dv_processing as dv
from datetime import timedelta

In [46]:
# Read event stream from file and split into event packets
recording = dv.io.MonoCameraRecording(AEDAT)
recording.getNextEventBatch()

min_batch_size = np.inf
max_batch_size = 0

if recording.isEventStreamAvailable():
    events_packets = []
    while True:
        events = recording.getNextEventBatch()
        if events is None:
            break
        
        if min_batch_size > len(events.numpy()):
            min_batch_size = len(events.numpy())
        
        if max_batch_size < len(events.numpy()):
            max_batch_size = len(events.numpy())
        
        events_packets.append(pd.DataFrame(events.numpy()))
        print(f"Number of events in Event Batch: {len(events.numpy())}")

    print("\n-----")
    print("Total Event Batches:", len(events_packets))
    print("Minimum Batch Size:", min_batch_size)
    print("Maximum Batch Size:", max_batch_size)

Number of events in Event Batch: 125
Number of events in Event Batch: 127
Number of events in Event Batch: 160
Number of events in Event Batch: 81
Number of events in Event Batch: 101
Number of events in Event Batch: 97
Number of events in Event Batch: 105
Number of events in Event Batch: 190
Number of events in Event Batch: 129
Number of events in Event Batch: 159
Number of events in Event Batch: 170
Number of events in Event Batch: 243
Number of events in Event Batch: 254
Number of events in Event Batch: 357
Number of events in Event Batch: 483
Number of events in Event Batch: 639
Number of events in Event Batch: 873
Number of events in Event Batch: 1191
Number of events in Event Batch: 1347
Number of events in Event Batch: 1736
Number of events in Event Batch: 2090
Number of events in Event Batch: 2369
Number of events in Event Batch: 2672
Number of events in Event Batch: 2936
Number of events in Event Batch: 2988
Number of events in Event Batch: 3153
Number of events in Event Batch